In [1]:
import os, fnmatch
import matplotlib as mpl
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from matplotlib.font_manager import FontProperties
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from matplotlib.colors import Normalize
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.externals.six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import optunity
import optunity.metrics

ModuleNotFoundError: ignored

### Input the reaction parameter constraints for the 3D space

In [ ]:
file = "perovskitedata_constrain_morph.csv" # convex vertices of allowed experimental space for morph-Pb-I
df=pd.read_csv(file)
Inchi = pd.read_csv("Organic Inchikey.csv")
Inchidict = dict(zip(Inchi['Chemical Name'], Inchi['InChI Key (ID)']))

Amine_done = []
for i in df['_rxn_organic_inchikey']:
    if i not in Amine_done: 
        Amine_done.append(i)
Amine_number = len(Amine_done)
for i in range(Amine_number):
    Amine_done[i] = dict(zip(Inchi['InChI Key (ID)'],Inchi['Chemical Name']))[Amine_done[i]]
print (Amine_done)
print ('*'*120)
print ('Number of amines:', Amine_number)

In [ ]:
# function for specific amine, crystal class
def per3D(points, amine = None, category = None, \
          dim = ['_rxn_M_inorganic', '_rxn_M_organic', '_rxn_M_acid']):
    if amine is not None: amine = Inchidict[amine]
    if amine is None:
        if category is None:
            All_points = np.zeros((points.shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]]
            All_points[:,3] = points['_out_crystalscore']
        else:
            All_points = np.zeros((points[points['_out_crystalscore'] == category].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points['_out_crystalscore'] == category]
            All_points[:,3] = points['_out_crystalscore'][points['_out_crystalscore'] == category]    
    else:
        if category is None:
            All_points = np.zeros((points[points["_rxn_organic_inchikey"] == amine].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points["_rxn_organic_inchikey"] == amine]
            All_points[:,3] = points['_out_crystalscore'][points["_rxn_organic_inchikey"] == amine]
        else:
            All_points = np.zeros((points[points["_rxn_organic_inchikey"] == amine][points['_out_crystalscore'] == category].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points["_rxn_organic_inchikey"] == amine][points['_out_crystalscore'] == category]
            All_points[:,3] = points['_out_crystalscore'][points["_rxn_organic_inchikey"] == amine][points['_out_crystalscore'] == category]
    
    return All_points

    
# Check if point is in convexhull
def point_in_hull(point, hull, tolerance = 1e-12):
    return all((np.dot(equ[:-1],point) + equ[-1] <= tolerance) for equ in hull.equations)


# Generate meshgrid points of certain size and location
def gridgen(n=20,x=[0,3],y=[0,4],z=[0,10], plot = False):
    a = np.linspace(x[0],x[1],n)
    b = np.linspace(y[0],y[1],n)
    c = np.linspace(z[0],z[1],n)
    points = np.zeros((n*n*n,3))
    xv,yv,zv = np.meshgrid(a,b,c)
    if plot: 
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(xv, yv, zv, c = 'coral', s = 5)
    for i in tqdm(range(n*n*n)):
        points[i] = [xv.flatten()[i], yv.flatten()[i], zv.flatten()[i]]
    return points

### Generate and plot the convexhull for allowed reaction-composition space in 3D

In [ ]:
amine_name = "Morpholinium Iodide" # The ammonium you would like to plot
point_3 = per3D(df,amine = amine_name,category = 3)
point_4 = per3D(df,amine = amine_name, category = 4)
point = np.vstack((point_3,point_4))

convexhull = ConvexHull(point[:,0:3])

xlim = [0,2.5]
ylim = [0,3]
zlim = [0,16]
view = [-143,13]

%matplotlib notebook
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(111, projection='3d')

n_facet = np.shape(convexhull.simplices)[0]
facets = np.zeros((n_facet,3,3))


for i in range(n_facet):
    for j in range(3):
        facets[i][j] = point[convexhull.simplices[i][j],0:3]
        
from itertools import combinations
comb = list(combinations([0,1,2],2))

for i in range(n_facet):
    for j in list(comb):
        x = [facets[i][j[0]][0],facets[i][j[1]][0]]
        y = [facets[i][j[0]][1],facets[i][j[1]][1]]
        z = [facets[i][j[0]][2],facets[i][j[1]][2]]
        verts = [list(zip(x, y, z))]
        ax.add_collection3d(Line3DCollection(verts, colors='black', linewidths=0.5))
    
ax.set_xlim(xlim[0],xlim[1])
ax.set_ylim(ylim[0],ylim[1])
ax.set_zlim(zlim[0],zlim[1])
ax.view_init(azim = view[0],elev = view[1])
plt.savefig('Graphs/convexhull of reaction composition space'+amine_name+'.svg', format = 'svg', transparent = True, dpi = 1000)

### Generate reactions inside the convex hull.

In [ ]:
vertices = convexhull.points[convexhull.vertices]
x_range = [min(vertices[:,0]),max(vertices[:,0])]
y_range = [min(vertices[:,1]),max(vertices[:,1])]
z_range = [min(vertices[:,2]),max(vertices[:,2])]

# Generate meshgrid point
grid_points = gridgen(n=6,x=x_range,y=y_range,z=z_range)
print("Finished generating grid points")
print("*"*20)

# Search and plot meshgrid points in class4 hull and total hull (also find meshgrid point in other hulls)
point_in_box = []

for j in tqdm(grid_points):
    if point_in_hull(j, convexhull):
        point_in_box.append(list(j))
point_in_box = np.array(point_in_box)

%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(point_in_box[:,0], point_in_box[:,1], point_in_box[:,2], c = 'gray', alpha = 0.5, s = 10)

from itertools import combinations
comb = list(combinations([0,1,2],2))

for i in range(n_facet):
    for j in list(comb):
        x = [facets[i][j[0]][0],facets[i][j[1]][0]]
        y = [facets[i][j[0]][1],facets[i][j[1]][1]]
        z = [facets[i][j[0]][2],facets[i][j[1]][2]]
        verts = [list(zip(x, y, z))]
        ax.add_collection3d(Line3DCollection(verts, colors='red', linewidths=1))

ax.set_xlim(xlim[0],xlim[1])
ax.set_ylim(ylim[0],ylim[1])
ax.set_zlim(zlim[0],zlim[1])
ax.view_init(azim = view[0],elev = view[1])
print("number of grid points generated in the hull:", point_in_box.shape[0])

# Function to save any obj in 'obj' folder
import pickle
def save_obj(obj, name ):
    with open('../RR Paper/'+ name + '.pkl', 'wb') as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('../RR Paper/' + name + '.pkl', 'rb') as file:
        return pickle.load(file)

# save_obj(point_in_box, 'Statespace_10X10X10_>0.1PbI2')

In [ ]:
import kennardstone

## select a subgroup of points using Kennard-Stone algorithm from 
sample_idx, _ = kennardstone.kennardstonealgorithm(point_in_box, 48)
sample_lst = list()
for i in sample_idx:
    sample_lst.append(list(point_in_box[i]))

sample_lst = np.array(sample_lst)
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(sample_lst[:,0], sample_lst[:,1], sample_lst[:,2], c = 'blue', alpha = 1, s = 10)
from itertools import combinations
comb = list(combinations([0,1,2],2))

for i in range(n_facet):
    for j in list(comb):
        x = [facets[i][j[0]][0],facets[i][j[1]][0]]
        y = [facets[i][j[0]][1],facets[i][j[1]][1]]
        z = [facets[i][j[0]][2],facets[i][j[1]][2]]
        verts = [list(zip(x, y, z))]
        ax.add_collection3d(Line3DCollection(verts, colors='red', linewidths=1))

ax.set_xlim(xlim[0],xlim[1])
ax.set_ylim(ylim[0],ylim[1])
ax.set_zlim(zlim[0],zlim[1])
ax.view_init(azim = view[0],elev = view[1])

In [ ]:
## Calcuate reagent volumes from concentration
# input stock solution concentration here
R_2_A = 2.32 # reagent2 Pb2+ concentration in M
R_2_B = 2.91 # reagent2 A+ concentration in M
R_3 = 2.36 # reagent3 A+ concentration in M
tot_vol = 500 # total volume of target solution in microLiter

V2 = (tot_vol*sample_lst[:,0])/R_2_A # reagent 2 volume
V3 = (tot_vol*sample_lst[:,1]-V2*R_2_B)/R_3 # reagent 3 volume
VFAH = ((tot_vol*sample_lst[:,2])/1000)*46/1.22 # reagent 4 volume
V1 = tot_vol-V2-V3-VFAH # reagent 1 volume
V4 = VFAH/2 # formic acid addition 1 volume
V5 = VFAH/2 # formic acid addition 2 volume

V1 = V1.round()
V2 = V2.round()
V3 = V3.round()
V4 = V4.round()
V5 = V5.round()

In [ ]:
V1[V1<0] = 0 # Replace negative volumes: we assume the negative volumes are from convex hull accuracy
# CAUTION: if V1 is too negative, something else is wrong.

In [ ]:
all(V1>=0)

In [ ]:
V1

In [ ]:
# Generate new robot input files

df_rbtinput=pd.read_excel('RobotInput_template.xls')

pipette_low = "Tip_50ul_Water_DispenseJet_Empty"
pipette_med = "StandardVolume_Water_DispenseJet_Empty"
pipette_high = "HighVolume_Water_DispenseJet_Empty"

df_rbtinput['Reagent1 (ul)'] = np.hstack((V1,V1))
df_rbtinput['Reagent2 (ul)'] = np.hstack((V2,V2))
df_rbtinput['Reagent3 (ul)'] = np.hstack((V3,V3))
df_rbtinput['Reagent6 (ul)'] = np.hstack((V4,V4))
df_rbtinput['Reagent7 (ul)'] = np.hstack((V5,V5))
df_rbtinput['Parameter Values'][0] = 105 # reaction temperature
df_rbtinput['Parameter Values'][1] = 750 # stir rate
df_rbtinput['Parameter Values'][2] = 900 # mixing time 1
df_rbtinput['Parameter Values'][3] = 1200 # mixing time 2
df_rbtinput['Parameter Values'][4] = 21600 # reaction time
df_rbtinput['Parameter Values'][5] = 80 # Preheat temperature

def liquid_class (vol):
    if max(vol) <= 50:
        output = pipette_low
    elif (max(vol) > 50) & (max(vol) <= 300):
        output = pipette_med
    elif (max(vol) > 300) & (max(vol) <= 1000):
        output = pipette_high
    else:
        raise ValueError("ValueError of volume given")
    
    return output
    
df_rbtinput['Liquid Class'][0] = liquid_class(df_rbtinput['Reagent1 (ul)'])
df_rbtinput['Liquid Class'][1] = liquid_class(df_rbtinput['Reagent2 (ul)'])
df_rbtinput['Liquid Class'][2] = liquid_class(df_rbtinput['Reagent3 (ul)'])
df_rbtinput['Liquid Class'][3] = liquid_class(df_rbtinput['Reagent4 (ul)'])
df_rbtinput['Liquid Class'][4] = liquid_class(df_rbtinput['Reagent5 (ul)'])
df_rbtinput['Liquid Class'][5] = liquid_class(df_rbtinput['Reagent6 (ul)'])
df_rbtinput['Liquid Class'][6] = liquid_class(df_rbtinput['Reagent7 (ul)'])
df_rbtinput['Liquid Class'][7] = liquid_class(df_rbtinput['Reagent8 (ul)'])
df_rbtinput['Liquid Class'][8] = liquid_class(df_rbtinput['Reagent9 (ul)'])

df_rbtinput.to_csv("new run for humidity_3and4 convex_"+amine_name+"_robotinput.csv")